In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import pandas as pd 
import os
import time
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

from model import lstm, aagcn, stconv, SAM
from data.handpose_dataset import HandPoseDatasetNumpy, df_to_numpy
from data.get_data_from_csv import get_train_data, get_val_data
from config import CFG
from utils import training_supervision, adj_mat

In [2]:
device = torch.device("cuda")

In [15]:
df = pd.read_csv("data/graphdata/custom_data/test_all_gestures.csv")
df = df.drop("Unnamed: 0", axis=1)
df.head()

,WRIST,THUMB_CMC,THUMB_MCP,THUMB_IP,THUMB_TIP,INDEX_FINGER_MCP,INDEX_FINGER_PIP,INDEX_FINGER_DIP,INDEX_FINGER_TIP,MIDDLE_FINGER_MCP,...,MIDDLE_FINGER_TIP,RING_FINGER_MCP,RING_FINGER_PIP,RING_FINGER_DIP,RING_FINGER_TIP,PINKY_MCP,PINKY_PIP,PINKY_DIP,PINKY_TIP,LABEL
0,"(0.49, 0.93, 0.23)","(0.46, 0.92, 0.23)","(0.41, 0.93, 0.23)","(0.38, 0.94, 0.23)","(0.36, 0.94, 0.23)","(0.40, 0.95, 0.27)","(0.38, 0.96, 0.30)","(0.37, 0.97, 0.31)","(0.35, 0.97, 0.32)","(0.42, 0.95, 0.29)",...,"(0.37, 0.96, 0.35)","(0.43, 0.95, 0.30)","(0.41, 0.95, 0.33)","(0.40, 0.96, 0.34)","(0.39, 0.96, 0.36)","(0.45, 0.94, 0.31)","(0.44, 0.94, 0.33)","(0.43, 0.94, 0.34)","(0.42, 0.94, 0.35)",Grasp
1,"(0.49, 0.93, 0.23)","(0.46, 0.92, 0.23)","(0.41, 0.93, 0.23)","(0.38, 0.94, 0.23)","(0.36, 0.94, 0.23)","(0.40, 0.95, 0.27)","(0.38, 0.96, 0.30)","(0.37, 0.96, 0.31)","(0.35, 0.97, 0.32)","(0.42, 0.95, 0.29)",...,"(0.37, 0.96, 0.35)","(0.43, 0.95, 0.30)","(0.41, 0.95, 0.33)","(0.40, 0.96, 0.34)","(0.39, 0.96, 0.36)","(0.45, 0.94, 0.31)","(0.44, 0.94, 0.33)","(0.43, 0.94, 0.34)","(0.42, 0.94, 0.35)",Grasp
2,"(0.49, 0.93, 0.23)","(0.46, 0.92, 0.23)","(0.41, 0.93, 0.23)","(0.38, 0.94, 0.23)","(0.36, 0.94, 0.23)","(0.40, 0.95, 0.27)","(0.38, 0.96, 0.30)","(0.37, 0.97, 0.31)","(0.35, 0.97, 0.32)","(0.42, 0.95, 0.29)",...,"(0.37, 0.96, 0.35)","(0.43, 0.95, 0.30)","(0.41, 0.95, 0.33)","(0.40, 0.96, 0.34)","(0.39, 0.96, 0.36)","(0.45, 0.94, 0.31)","(0.44, 0.94, 0.33)","(0.43, 0.94, 0.34)","(0.42, 0.94, 0.35)",Grasp
3,"(0.49, 0.92, 0.23)","(0.46, 0.92, 0.23)","(0.41, 0.93, 0.23)","(0.38, 0.93, 0.23)","(0.36, 0.93, 0.23)","(0.40, 0.95, 0.27)","(0.38, 0.96, 0.30)","(0.37, 0.96, 0.31)","(0.35, 0.96, 0.33)","(0.42, 0.95, 0.29)",...,"(0.37, 0.96, 0.35)","(0.43, 0.94, 0.30)","(0.41, 0.95, 0.33)","(0.40, 0.95, 0.35)","(0.39, 0.95, 0.36)","(0.45, 0.93, 0.31)","(0.44, 0.94, 0.33)","(0.43, 0.93, 0.34)","(0.42, 0.93, 0.35)",Grasp
4,"(0.49, 0.92, 0.24)","(0.46, 0.92, 0.23)","(0.41, 0.93, 0.23)","(0.38, 0.93, 0.23)","(0.36, 0.93, 0.23)","(0.40, 0.94, 0.28)","(0.38, 0.95, 0.30)","(0.37, 0.96, 0.32)","(0.35, 0.96, 0.33)","(0.42, 0.94, 0.29)",...,"(0.37, 0.95, 0.36)","(0.43, 0.94, 0.30)","(0.41, 0.94, 0.33)","(0.40, 0.94, 0.35)","(0.39, 0.94, 0.36)","(0.45, 0.93, 0.31)","(0.44, 0.93, 0.33)","(0.43, 0.93, 0.35)","(0.42, 0.92, 0.35)",Grasp


In [16]:
data_numpy = df_to_numpy(df)
data_numpy

(array([[0.49, 0.93, 0.23, ..., 0.42, 0.94, 0.35],
        [0.49, 0.93, 0.23, ..., 0.42, 0.94, 0.35],
        [0.49, 0.93, 0.23, ..., 0.42, 0.94, 0.35],
        ...,
        [0.47, 1.12, 0.28, ..., 0.41, 1.14, 0.39],
        [0.47, 1.12, 0.28, ..., 0.41, 1.14, 0.39],
        [0.47, 1.12, 0.28, ..., 0.41, 1.14, 0.39]]),
 array([[1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1]]))

In [17]:
test_set_2 = HandPoseDatasetNumpy(data_numpy, distances=True)
test_loader_2 = DataLoader(test_set_2, batch_size=CFG.batch_size, drop_last=True)
graph = aagcn.Graph(adj_mat.num_node, adj_mat.self_link, adj_mat.inward, adj_mat.outward, adj_mat.neighbor)
model_2 = aagcn.Model(num_class=CFG.num_classes, num_point=21, num_person=1, graph=graph, drop_out=0.5, in_channels=3)
print(f"[INFO] TESTING ON {len(test_set_2)} DATAPOINTS")

[INFO] TESTING ON 192 DATAPOINTS


In [18]:
class FocalLoss(nn.Module):
    
    def __init__(self, weight=None, 
                 gamma=2., reduction='mean'):
        nn.Module.__init__(self)
        self.weight = weight
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, input_tensor, target_tensor):
        log_prob = F.log_softmax(input_tensor, dim=-1)
        prob = torch.exp(log_prob)
        return F.nll_loss(
            ((1 - prob) ** self.gamma) * log_prob, 
            target_tensor, 
            weight=self.weight,
            reduction = self.reduction
        )

In [19]:
def eval_func(model, criterion, data_loader, epoch):
    model.eval()
    preds = []
    groundtruth = []
    t0 = time.time()
    loss_total = 0
    global_step = 0
    iters = len(data_loader)
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(data_loader):
            labels = labels.cuda().long()
            inputs = inputs.cuda().float()

            last_label = labels[:, -1, :]
            last_label = torch.argmax(last_label, 1)

            last_out = model(inputs)
            loss = criterion(last_out, last_label)

            preds.append(last_out.cpu().detach().numpy())
            groundtruth.append(last_label.cpu().detach().numpy())
            loss_total += loss

            if i%CFG.print_freq == 1 or i == iters-1:
                t1 = time.time()
                print(f"[EVAL] Iteration: {i}/{iters} | Val-Loss: {loss_total/i} | ETA: {((t1-t0)/i * iters) - (t1-t0)}s")

    return loss_total, np.array(preds),  np.array(groundtruth).flatten()

In [20]:
MODEL_PATH_2 = "trained_models/7_AAGCN_Focal_seqlen32_release_SAM_joints1_joints2_oridist/f10.8142688679245284_valloss310.2437744140625_epoch13.pth"


In [21]:
model_2.load_state_dict(torch.load(MODEL_PATH_2)["model_state_dict"])
model_2.cuda()

criterion = FocalLoss()

In [22]:
val_loss_2, preds_val_2, gt_val_2 = eval_func(model_2, criterion, test_loader_2, 0)
print(f"[EVAL] VALIDATION LOSS MODEL 2 {val_loss_2}")
print(classification_report(gt_val_2, np.argmax(preds_val_2, axis=2).flatten(), target_names=CFG.classes, digits=4))


[EVAL] Iteration: 1/12 | Val-Loss: 3.5996451377868652 | ETA: 2.7948360443115234s
[EVAL] Iteration: 11/12 | Val-Loss: 2.3618175983428955 | ETA: 0.05342555046081543s
[EVAL] VALIDATION LOSS MODEL 2 25.97999382019043
              precision    recall  f1-score   support

       Grasp     0.0000    0.0000    0.0000         1
        Move     0.0000    0.0000    0.0000        32
    Negative     0.2022    0.5625    0.2975        32
    Position     0.3409    0.4688    0.3947        32
       Reach     0.7143    0.3125    0.4348        32
     Release     0.0000    0.0000    0.0000        63

    accuracy                         0.2240       192
   macro avg     0.2096    0.2240    0.1878       192
weighted avg     0.2096    0.2240    0.1878       192


C:\Users\smujan\PycharmProjects\Hand-Gesture-Recognition-in-manual-assembly-tasks-using-GCN\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\smujan\PycharmProjects\Hand-Gesture-Recognition-in-manual-assembly-tasks-using-GCN\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\smujan\PycharmProjects\Hand-Gesture-Recognition-in-manual-assembly-tasks-using-GCN\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and bein